# Make a simple Flask site that streams video from the camera

In [ ]:
from flask import Flask, render_template, Response

app = Flask(__name__)

import cv2
import numpy as np
import pyrealsense2 as rs

def gen():
    frame_width = 640
    frame_height = 480
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_stream(rs.stream.color, frame_width, frame_height, rs.format.bgr8, 30)
    config.enable_stream(rs.stream.depth, frame_width, frame_height, rs.format.z16, 30)

    
    # Create an align object
    align_to = rs.stream.color
    align = rs.align(align_to)

    pipeline.start(config)
    try:
        while True:
            frames = pipeline.wait_for_frames()
            # Align the depth frame to color frame
            aligned_frames = align.process(frames)

            # Get aligned frames
            depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
            color_frame = aligned_frames.get_color_frame()
            if not depth_frame or not color_frame:
                continue
                
            # Convert images to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())
            
            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

            
            #Do face detection on RGB
            gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                color_image = cv2.rectangle(color_image,(x,y),(x+w,y+h),(255,0,0),2)
                # get the rectangle img around all the faces
                sub_face = color_image[y:y+h, x:x+w]
                # apply a gaussian blur on this new recangle image
                sub_face = cv2.GaussianBlur(sub_face,(23, 23), 30)
                # merge this blurry rectangle to our final image
                color_image[y:y+sub_face.shape[0], x:x+sub_face.shape[1]] = sub_face
            
            # Stack both images horizontally
            images = np.hstack((color_image, depth_colormap))
            
            #Downsample
            #images = images[::5,::5]
            
            #Encode and deploy
            ret, jpeg = cv2.imencode('.jpg', images)
            frame = jpeg.tobytes()
            #yield frame
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')
            
    finally:

        # Stop streaming
        pipeline.stop()

@app.route('/')
def video_feed():
    return Response(gen(),
                    #mimetype='image/jpeg')#
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jan/2020 19:05:13] "GET / HTTP/1.1" 200 -


# To get this flask server streaming:
Reverse port forward to the aicare server
`ssh -v -R 50051:localhost:5000 danjwu@aicare.stanford.edu`

Port forward from client to aicare server
`ssh -L 50051:localhost:50051 danjwu@aicare.stanford.edu`
        
Visit localhost:50051 on client